In [1]:
import torch
from torch import nn
from torch.nn import init

# pytorch已进行默认初始化
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [2]:
print(type(net.named_parameters()))

for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [3]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [4]:
#返回的param的类型为torch.nn.parameter.Parameter
#其实这是Tensor的子类，Tensor拥有的属性它都有
#和Tensor不同的是，如果一个Tensor是Parameter，它会自动被添加到模型的参数列表里

class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [5]:
#正态分布

for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 1.3783e-02,  3.5607e-03,  8.0120e-03,  3.3789e-05],
        [-8.6220e-03, -5.2619e-03, -1.0405e-02,  6.5414e-03],
        [ 4.8574e-03,  8.9518e-03, -1.4520e-02, -5.8699e-03]])
2.weight tensor([[-0.0010,  0.0173,  0.0054]])


In [6]:
#使用常数初始化

for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


In [7]:
#自定义初始化方法，令权重有一半概率初始化为0，另一半概率初始化为[-10，-5]，[5，10]区间里均匀分布的随机数

def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-6.2757,  0.0000, -7.7713,  0.0000],
        [-5.3274,  8.0216, -7.8206,  5.5378],
        [ 0.0000, -0.0000, -0.0000,  0.0000]])
2.weight tensor([[-0.0000, -0.0000, -7.7197]])


In [8]:
#还可以改变参数的data来改写模型参数值同时不影响梯度

for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


In [9]:
#共享模型参数

linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [10]:
#在内存中这两个线性层其实是一个对象

print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [11]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) # 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
